# SDK and pandas demo

This notebook demonstrates the relation between pandas and the python SDK

In [1]:
from matplotlib import pyplot as plt
import os
from datetime import datetime
from getpass import getpass
from cognite.client.experimental import CogniteClient
from cognite.client.exceptions import *
project = 'publicdata'
api_key = os.environ['PUBLICDATA_API_KEY'] or getpass("Open Industrial Data API-KEY: ")
client = CogniteClient(project=project,api_key=api_key,client_name="STS demo")

## The SDK has several built-in types to represent CDF data

In [2]:
ts = client.time_series.list(limit=10)
ts.__class__

cognite.client.data_classes.time_series.TimeSeriesList

In [3]:
ts[0].__class__

cognite.client.data_classes.time_series.TimeSeries

## Since late 2019, these classes are shown by default in a readable, pandas-like way in notebooks

In [4]:
ts

,id,external_id,name,is_string,metadata,asset_id,is_step,description,created_time,last_updated_time
0,35246780681261,VAL_23-LY-92529_SILch0_SC0_TYPSP:VALUE,VAL_23-LY-92529_SILch0_SC0_TYPSP:VALUE,False,{'tag': 'VAL_23-LY-92529_SILch0_SC0_TYPSP:VALU...,3424990723231138,False,PH (Profiler) Shutdown Channel 0 - Tube y Posi...,0,0
1,41852231325889,VAL_45-FT-92139B:X.Value,VAL_45-FT-92139B:X.Value,False,"{'tag': 'VAL_45-FT-92139B:X.Value', 'scan': '1...",4293345866058133,False,PH 1stStg LO Cooler CW,0,0
2,44435358976768,VAL_23-LT-96182-03:X.Value,VAL_23-LT-96182-03:X.Value,False,"{'tag': 'VAL_23-LT-96182-03:X.Value', 'scan': ...",5826132550305704,False,PH 1stStg Lube Oil Reservoir,0,0
3,81505042780431,VAL_23-TT-96103-03:Z.X.Value,VAL_23-TT-96103-03:Z.X.Value,False,"{'tag': 'VAL_23-TT-96103-03:Z.X.Value', 'scan'...",7271158805875629,False,PH 1stStg Comp Thrust BRG IN,0,0
4,82185797798532,VAL_23-KA-9101-M01_OC_instantaneous_residual_N...,VAL_23-KA-9101-M01_OC_instantaneous_residual_N...,False,{'tag': 'VAL_23-KA-9101-M01_OC_instantaneous_r...,6191827428964450,False,PH (SwitchGear) MV-COMP.M. FEEDER/ 51-3 Non D...,0,0
5,86240763048067,VAL_23-KA-9101-M01-EL:XS.CoreStatus.Xservice.S...,VAL_23-KA-9101-M01-EL:XS.CoreStatus.Xservice.S...,False,{'tag': 'VAL_23-KA-9101-M01-EL:XS.CoreStatus.X...,6191827428964450,False,PH Truck in Service,0,0
6,94387769425302,VAL_23-KA-9101-M01_Motor_startup_supervision_M...,VAL_23-KA-9101-M01_Motor_startup_supervision_M...,False,{'tag': 'VAL_23-KA-9101-M01_Motor_startup_supe...,6191827428964450,False,PH (SwitchGear),0,0
7,160363870036631,VAL_23_PT_92537:Z.X.Value,VAL_23_PT_92537:Z.X.Value,False,"{'tag': 'VAL_23_PT_92537:Z.X.Value', 'scan': '...",1543828113500234,False,PH 1stStgComp Discharge,0,0
8,161590978537613,VAL_23-PT-92523:X.Value,VAL_23-PT-92523:X.Value,False,"{'tag': 'VAL_23-PT-92523:X.Value', 'scan': '1'...",6990527136674720,False,PH 1stStgSuctScrubber,0,0
9,171465662628809,VAL_23-YA-96120-01:Z.X.Value,VAL_23-YA-96120-01:Z.X.Value,False,"{'tag': 'VAL_23-YA-96120-01:Z.X.Value', 'scan'...",2697126585129848,False,PH 1stStg Gear2 Journ BRG DE,0,0


In [5]:
ts[0]

,value
id,35246780681261
external_id,VAL_23-LY-92529_SILch0_SC0_TYPSP:VALUE
name,VAL_23-LY-92529_SILch0_SC0_TYPSP:VALUE
is_string,False
asset_id,3424990723231138
is_step,False
description,PH (Profiler) Shutdown Channel 0 - Tube y Posi...
created_time,0
last_updated_time,0
tag,VAL_23-LY-92529_SILch0_SC0_TYPSP:VALUE


## Before this, a JSON like format was used

In [6]:
print(str(ts[0]))

{
    "id": 35246780681261,
    "external_id": "VAL_23-LY-92529_SILch0_SC0_TYPSP:VALUE",
    "name": "VAL_23-LY-92529_SILch0_SC0_TYPSP:VALUE",
    "is_string": false,
    "metadata": {
        "tag": "VAL_23-LY-92529_SILch0_SC0_TYPSP:VALUE",
        "scan": "1",
        "span": "100.0",
        "step": "0",
        "zero": "0.0",
        "recno": "145073",
        "excdev": "0.0",
        "excmax": "600",
        "excmin": "0",
        "exdesc": "",
        "future": "0",
        "compdev": "0.0",
        "compmax": "28800",
        "compmin": "0",
        "convers": "1.0",
        "pointid": "160573",
        "srcptid": "0",
        "engunits": "",
        "shutdown": "0",
        "userint1": "0",
        "userint2": "0",
        "archiving": "1",
        "location1": "1",
        "location2": "0",
        "location3": "1",
        "location4": "1",
        "location5": "0",
        "pointtype": "12",
        "ptclassid": "2",
        "sourcetag": "",
        "totalcode": "0",
       

## But keep in mind, these built-in types do not behave like pandas dataframes

In [7]:
ts.loc[:,'external_id']

AttributeError: 'TimeSeriesList' object has no attribute 'loc'

## Instead they behave like python objects and lists, and you can use list comprehensions and iterators on them

In [ ]:
[t.external_id for t in ts]

## You can convert built-in types using the to_pandas method

In [ ]:
ts.to_pandas()

### See the difference? to indicate what fields exist, we do not camelCase output of built-in types, but follow this convention only when a pandas dataframe is explicitly requested

In [ ]:
ts.to_pandas(camel_case=False) # you can still override this

### Now you can use pandas methods on the dataframes, although in this case, the list comprehension method is recommended

In [ ]:
df = ts.to_pandas()
df.loc[:,'externalId'].values

## An exception to this is datapoint retrieval, which can return dataframes directly with the retrieve_dataframe method

In [ ]:
df = client.datapoints.retrieve_dataframe(external_id=[t.external_id for t in ts[:3]],
                                          start='4w-ago',end='now',granularity='1d',aggregates=['average'],
                                         include_aggregate_name=False)
df

## Of course, using a built-in datatype is also possible here

In [ ]:
dpt = client.datapoints.retrieve(id=[t.id for t in ts[:3]],
                                start='4w-ago',end='now',granularity='1d',aggregates=['average'])
print(dpt.__class__,dpt[1].__class__)

In [ ]:
dpt[1] # I am not a dataframe

### These objects have .timestamp, .value and .[aggregate] fields to use directly

In [ ]:
[(t,d) for t,d in zip(dpt[1].timestamp[:10], dpt[1].average[:10])]